In [3]:
import os
import argparse
import time

import matplotlib as mpl
mpl.use('TkAgg') # sets the backend for matplotlib
import matplotlib.pyplot as plt

from utils.redraw_obstacles import redraw_obstacles, set_non_blocking
from commonroad.common.file_reader import CommonRoadFileReader
from commonroad.visualization.draw_dispatch_cr import draw_object

import numpy as np


In [4]:
file_path = '/home/rong/VAE-Motion-Planning/scenarios/commonroad_data/highway-ramp.cr.xml'
scenario, _ = CommonRoadFileReader(file_path).open()

In [87]:
def show_lane(scenario):
    plt.figure(figsize=(250, 100))
    draw_params={'time_begin':0}
    handles = {}
    draw_object(scenario, handles=handles, draw_params=draw_params)
    plt.gca().autoscale()
    plt.gca().set_aspect('equal')
    plt.show()

In [90]:
show_lane(scenario)

In [83]:
def plt_window(scenario, duration, draw_params, size, pos):
    set_non_blocking()  # ensures interactive plotting is activated
    figsize = size
    fig = plt.figure(figsize=(figsize[0] / inch_in_cm, figsize[1] / inch_in_cm))
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    # ax.set_facecolor("black")
    # ax.set_cmap("binary")
    fig.add_axes(ax)
    fig.patch.set_facecolor('black')
    plt.xlim(pos[0] - size[0]/2, pos[0] + size[0]/2)
    plt.ylim( pos[1] -size[1]/2, pos[1] + size[1]/2)
    plt.set_cmap("plasma")
    handles = {}  # collects handles of obstacle patches, plotted by matplotlib
    plot_limits = [pos[0] - size[0]/2, pos[0] + size[0]/2, pos[1] +-size[1]/2, pos[1] + size[1]/2]
    draw_object(scenario, handles=handles, draw_params=draw_params, plot_limits=plot_limits)
    # plt.gray()
    fig.canvas.draw()
    for handles_i in handles:
        if not handles_i:
            handles.pop()

    data = np.zeros(( (duration[1] - duration[0],) + fig.canvas.get_width_height()[::-1]), dtype='bool')

    t1 = time.time()
    # loop where obstacle positions are updated
    for i in range(duration[0], duration[1]):
        # ...
        # change positions of obstacles
        # ...
        draw_params['time_begin'] = i
        redraw_obstacles(scenario, handles=handles, figure_handle=fig, plot_limits=plot_limits, draw_params=draw_params)
        buffer = np.frombuffer(fig.canvas.renderer.buffer_rgba(), dtype=np.uint8).reshape(fig.canvas.get_width_height()[::-1] + (4,))
        buffer = buffer[:, : , 0].astype(bool)
        data[(i-duration[0]), :, : ] = buffer
        print('fps:', (i - duration[0])/(time.time()-t1))
    return data

In [85]:
size = [200, 40]
pos = [350, 135]
duration = [0, 100]
inch_in_cm = 10

data = plt_window(scenario=scenario, duration=duration, draw_params=draw_params, size=size, pos=pos)

fps: 0.0
fps: 9.566251870233186
fps: 13.056402434279132
fps: 15.452674548408607
fps: 16.659946635631787
fps: 17.60203319383663
fps: 18.45082921536138
fps: 17.964263752911684
fps: 18.686562210236435
fps: 19.34888984910436
fps: 19.751062354962176
fps: 19.819936636130294
fps: 20.438094255796322
fps: 20.97189910740694
fps: 21.388044528607296
fps: 21.61817733248759
fps: 21.79845813332398
fps: 21.867582794871865
fps: 21.714612122336455
fps: 21.545480254246684
fps: 21.27638398705451
fps: 21.482879015186974
fps: 21.694815280160892
fps: 22.019042436686888
fps: 22.177109240387278
fps: 21.80550194977175
fps: 20.52657104582418
fps: 19.782403395315615
fps: 19.668400651479438
fps: 19.620354162163558
fps: 19.116005782696757
fps: 18.848323265309908
fps: 18.255615036396236
fps: 17.939909121002724
fps: 17.83952397357866
fps: 17.577999337834385
fps: 17.21867214276787
fps: 16.928071654336243
fps: 16.595227494695017
fps: 16.3427943547384
fps: 16.31876001975695
fps: 16.38513646906912
fps: 16.517431411843237

In [59]:
def plt_data(data):
    fig = plt.figure()
    plt.gray()
    if len(data.shape) <=2:
        plt.imshow(data)
    else:
        myobj = plt.imshow(data[0, :, :])
        for row in data:
            myobj.set_data(row)
            plt.draw()
            plt.pause(0.01)

In [70]:
plt_data(data)

In [91]:
from commonroad.scenario.scenario import Scenario
new_scenario = Scenario(dt=0.1, benchmark_id=scenario.benchmark_id)
new_scenario.lanelet_network = scenario.lanelet_network
new_scenario.add_objects(scenario.dynamic_obstacles[0])
scenario.remove_obstacle(scenario.dynamic_obstacles[0])
show_lane(new_scenario)

In [86]:
data = plt_window(scenario=new_scenario, duration=duration, draw_params=draw_params, size=size, pos=pos)

fps: 0.0
fps: 10.271496581313794
fps: 15.450483022829621
fps: 19.472555158181766
fps: 22.48278468290395
fps: 23.933397622810972
fps: 25.644138470756552
fps: 26.59680024495039
fps: 26.8208873143968
fps: 27.118539644681356
fps: 27.985571861125827
fps: 28.911244138619036
fps: 28.73622855772607
fps: 29.273148468776657
fps: 29.43383031109899
fps: 29.82727886460696
fps: 30.10406916472018
fps: 30.738536177822077
fps: 31.091053301321516
fps: 31.546594219449986
fps: 31.84322788274585
fps: 32.272932258648396
fps: 32.52927485999409
fps: 32.45776593894162
fps: 32.75843479296572
fps: 32.77284423603864
fps: 33.1367055913606
fps: 33.31922296676057
fps: 33.149487334009834
fps: 32.76572264104488
fps: 32.450249522836415
fps: 32.44353197507588
fps: 32.18515481964186
fps: 31.98562435872549
fps: 32.036252823546725
fps: 31.94266012874925
fps: 31.698612100122265
fps: 31.699053823902005
fps: 31.850783861936385
fps: 32.09748911068893
fps: 32.2804889593183
fps: 32.23944631825901
fps: 32.431015447560235
fps: 32.

In [ ]:
def show_lane(scenario):
    plt.figure(figsize=(250, 100))
    draw_params={'time_begin':0}
    handles = {}
    draw_object(scenario, handles=handles, draw_params=draw_params)
    plt.gca().autoscale()
    plt.gca().set_aspect('equal')
    plt.show()